In [ ]:
 import time
import pandas as pd

!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
import sys
import urllib.request
import requests

from PIL import Image
from io import BytesIO
from selenium import webdriver


def fetch_image_urls(query: str, max_links_to_fetch: int, wd: webdriver, sleep_between_interactions: int = 1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)

    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"
    wd.get(search_url.format(q=query))
    print('search_url')
    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        for img in thumbnail_results[results_start:number_results]:
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    if (actual_image.get_attribute('src')[:5]=='https'):
                      image_urls.add(actual_image.get_attribute('src'))
                      print(image_urls)
            image_count = len(image_urls)
            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")
        results_start = len(thumbnail_results)
    return image_urls

print("Starting scrape from the given query files ")
print('df.index')
df = pd.read_csv('query.csv')
for ind in df.index:
    q = df['keyword'][ind]
    print("Selected keyword " + q)
    n = df['no'][ind]
    print("Number of pictures downloading for the keyword "+str(n))
    images = fetch_image_urls(q,n,wd)
    print("Image URLS fetched for the keyword")
    i = 0
    x = df['pwidth'][ind]
    y = df['pheight'][ind]
    print("The resolution of the images ("+ str(x)+","+str(y)+")")
    for url in images:
      print("hi")
      i = i+1
      print("hi2")
      response = requests.get(url)
      print("hi3")
      try: 
        img = Image.open(BytesIO(response.content))
        print("hi4")
        newsize = (x,y)
        print("hi5")
        img = img.resize(newsize)
        print("hi6")
        if img.mode != 'RGB':
          img = img.convert('RGB')
        img.save('MG'+q+"_"+str(i)+".jpeg","JPEG")
        print("hi"+str(i))
      except: 
        url=url[i+5]
    print("Download Complete for keyword")

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


Starting scrape from the given query files 
df.index
Selected keyword jackfruit
Number of pictures downloading for the keyword 100
search_url
Found: 100 search results. Extracting links from 0:100
{'https://health.clevelandclinic.org/wp-content/uploads/sites/3/2020/12/eatJackfruit-823672534-770x533-1.jpg'}
{'https://health.clevelandclinic.org/wp-content/uploads/sites/3/2020/12/eatJackfruit-823672534-770x533-1.jpg', 'https://upload.wikimedia.org/wikipedia/commons/b/ba/Jackfruit_hanging.JPG'}
{'https://health.clevelandclinic.org/wp-content/uploads/sites/3/2020/12/eatJackfruit-823672534-770x533-1.jpg', 'https://upload.wikimedia.org/wikipedia/commons/b/ba/Jackfruit_hanging.JPG', 'https://post.healthline.com/wp-content/uploads/2020/09/jackfruit-on-plate-thumb-732x549.jpg'}
{'https://health.clevelandclinic.org/wp-content/uploads/sites/3/2020/12/eatJackfruit-823672534-770x533-1.jpg', 'https://images.indianexpress.com/2020/09/jackfruit_1200_getty.jpg', 'https://upload.wikimedia.org/wikipedia/c

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


hi3
hi4
hi5
hi6
hi39
hi
hi2
hi3
hi4
hi5
hi6
hi40
hi
hi2
hi3
hi4
hi5
hi6
hi41
hi
hi2
hi3
hi4
hi5
hi6
hi42
hi
hi2
hi3
hi4
hi5
hi6
hi43
hi
hi2
hi3
hi4
hi5
hi6
hi44
hi
hi2
hi3
hi4
hi5
hi6
hi45
hi
hi2
hi3
hi4
hi5
hi6
hi46
hi
hi2
hi3
hi4
hi5
hi6
hi47
hi
hi2
hi3
hi4
hi5
hi6
hi48
hi
hi2
hi3
hi4
hi5
hi6
hi49
hi
hi2
hi3
hi4
hi5
hi6
hi50
hi
hi2
hi3
hi4
hi5
hi6
hi51
hi
hi2
hi3
hi4
hi5
hi6
hi52
hi
hi2
hi3
hi4
hi5
hi6
hi53
hi
hi2
hi3
hi4
hi5
hi6
hi54
hi
hi2
hi3
hi4
hi5
hi6
hi55
hi
hi2
hi3
hi4
hi5
hi6
hi56
hi
hi2
hi3
hi4
hi5
hi6
hi57
hi
hi2
hi3
hi4
hi5
hi6
hi58
hi
hi2
hi3
hi4
hi5
hi6
hi59
hi
hi2
hi3
hi4
hi5
hi6
hi60
hi
hi2
hi3
hi4
hi5
hi6
hi61
hi
hi2
hi3
hi4
hi5
hi6
hi62
hi
hi2
hi3
hi4
hi5
hi6
hi63
hi
hi2
hi3
hi4
hi5
hi6
hi64
hi
hi2
hi3
hi4
hi5
hi6
hi65
hi
hi2
hi3
hi4
hi5
hi6
hi66
hi
hi2
hi3
hi4
hi5
hi6
hi67
hi
hi2
hi3
hi4
hi5
hi6
hi68
hi
hi2
hi3
hi4
hi5
hi6
hi69
hi
hi2
hi3
hi4
hi5
hi6
hi70
hi
hi2
hi3
hi
hi2
hi3
hi4
hi5
hi6
hi72
hi
hi2
hi3
hi4
hi5
hi6
hi73
hi
hi2
hi3
hi4
hi5
hi6
hi74
hi
hi2
hi3
hi4
h

In [ ]:
! zip jackfruit.zip *.jpeg

  adding: MGjackfruit_10.jpeg (deflated 1%)
  adding: MGjackfruit_11.jpeg (deflated 1%)
  adding: MGjackfruit_12.jpeg (deflated 1%)
  adding: MGjackfruit_13.jpeg (deflated 1%)
  adding: MGjackfruit_14.jpeg (deflated 1%)
  adding: MGjackfruit_15.jpeg (deflated 2%)
  adding: MGjackfruit_16.jpeg (deflated 1%)
  adding: MGjackfruit_17.jpeg (deflated 1%)
  adding: MGjackfruit_18.jpeg (deflated 1%)
  adding: MGjackfruit_19.jpeg (deflated 1%)
  adding: MGjackfruit_1.jpeg (deflated 1%)
  adding: MGjackfruit_20.jpeg (deflated 1%)
  adding: MGjackfruit_21.jpeg (deflated 1%)
  adding: MGjackfruit_22.jpeg (deflated 1%)
  adding: MGjackfruit_23.jpeg (deflated 1%)
  adding: MGjackfruit_24.jpeg (deflated 2%)
  adding: MGjackfruit_25.jpeg (deflated 1%)
  adding: MGjackfruit_26.jpeg (deflated 2%)
  adding: MGjackfruit_27.jpeg (deflated 1%)
  adding: MGjackfruit_28.jpeg (deflated 5%)
  adding: MGjackfruit_29.jpeg (deflated 2%)
  adding: MGjackfruit_2.jpeg (deflated 2%)
  adding: MGjackfruit_30.jpeg (def